In [1]:
import sys
sys.path.insert(0,'/home/anonymous/trainjourney/nsb2/')
sys.path.insert(0,'/home/anonymous/src/blacksky/')

In [2]:
import numpy as np
import matplotlib.pyplot as plt

import histlite as hl
import astropy
from astropy.coordinates import SkyCoord, EarthLocation, AltAz
import astropy.units as u

In [7]:
from nsb.atmosphere import scattering, extinction

from nsb.emitter import moon, airglow, galactic, zodiacal

from nsb.core.logic import Layer, Scattering
from nsb.core import Frame, Model
#from nsb.core.emitter import Emitter
from nsb.core.instrument import Instrument, Camera, Pixel
from nsb.core.ray import Ray
from nsb.core.utils import reduce_rays, haversine, hist_sample
import nsb.utils.bandpass as bandpass

# Predicting Night Sky Background for H.E.S.S.

## 1. Configuring the model layers:

Configuring the models for emitters:

In [10]:
# Moonlight:
conf_jons = {}

# Diffuse Emitters:
conf_glow = {"H": 87}
conf_hmag  = {"Mag_0": 236.8*2.3504*1e-11}
conf_kawa = {'i100_max':50, 'star_percentage':0.25}

# Starlight
conf_gaia = {"catalog_file": '/lfs/l7/hess/users/gerritr/GaiaDR3Tycho.pkl', "magmin" : -2, "magmax": 15}

Configuring atmospheric scattering:

In [11]:
conf_mie = {"parameters": [0.8],
            "bins": [np.linspace(0, np.pi, 1000)]}      
conf_ray = {"parameters": [0.0148],
            "bins": [np.linspace(0, np.pi, 1000)]}

Creating the Instrument from a response object:

In [5]:
from nsb.instrument import RESPONSE_PATH
hess1u = Camera.from_response(RESPONSE_PATH+'hess1u_ct1.pkl')

## 2. Creating a model:
### Defining simple model including Moon, Stars and Airglow:

In [6]:
# Sources:
glow = airglow.Noll2012(conf_glow)
zodi = zodiacal.Masana2021({})

gaia = stars.StarCatalog(conf_gaia)
hmag = galactic.GaiaDR3Mag15(conf_hmag)
gbl  = galactic.Kawara2017(conf_kawa)

jons = moon.Jones2013(conf_jons)

# Atmospheric Extinction:
atm_airglow = extinction.Noll2012({'scale':1.6, 'offset':-0.16})([glow])
atm_diffuse = extinction.Masana2021({'gamma':0.5})([zodi, gbl])
atm_stellar = extinction.Masana2021({'gamma':1})([gaia, hmag])

# Atmospheric Scattering:
atm_ray = scattering.Rayleigh(conf_ray)([jons]).map(np.deg2rad(180))
atm_mie = scattering.Mie(conf_mie)([jons]).map(np.deg2rad(180))

# Camera:
CT1 = Instrument({'camera':hess1u, 'bandpass':bandpass.HESS1U()}, 1)([atm_stellar, atm_ray, atm_mie, atm_tes, atm_tet])

NameError: name 'airglow' is not defined

### Compiling the model:

In [ ]:
%%time
model = Model(CT1)
model.compile()

### Visualizing the pipeline

In [ ]:
print(model.summary())

## 3. Determining a frame to capture:
A "frame" describing a capture is determined by:
 - A location
 - An observation time in UTC
 - A target at which to point

In [ ]:
location = EarthLocation.from_geodetic(16.5028, -23.27280, 1800.)
obstime  = astropy.time.Time('2021-03-21T22:14:16', format='isot', scale='utc')
target   = SkyCoord.from_name('eta car')

frame = Frame(location, obstime, target, 3.2, -0.25*u.deg, np.linspace(270, 730, 30)*u.nm, albedo=0.85, aero=[0.065, 1.2], sfu=75)

## 4. Using the model to predict NSB

In [ ]:
%%time
res = model.predict(frame)

## 5. Visualizing the data:
Each pipeline layer has it's corresponding visualization. It can either be called on a frame or rays generated by the same layer. As such, the final data can be easily evaluated:

In [ ]:
from ctapipe.visualization import CameraDisplay
def ctapipe_disp(cam, instrument, rays, ax, label='a.u.', **kwargs):
    display = CameraDisplay(cam, ax=ax, **kwargs)
    display.image = instrument.camera.pix_assign(rays)
    display.add_colorbar(label=label)
    return display

In [ ]:
fig, ax = plt.subplots()
disp = ctapipe_disp(cam, CT1, res, ax, 'Rate [Hz]', show_frame=False)
disp.set_limits_minmax(0, 1.8e9)

### Individual contributions:

In [ ]:
fig, ax = plt.subplots(nrows=2, ncols=3, figsize=(20,10))
disp1 = ctapipe_disp(cam, CT1, res[res.source == type(jons)], ax[0,0], 'Rate [Hz]', title='Moon', show_frame=False)
disp2 = ctapipe_disp(cam, CT1, res[res.source == type(glow)], ax[0,1], 'Rate [Hz]', title='Airglow', show_frame=False)
disp3 = ctapipe_disp(cam, CT1, res[res.source == type(zodi)], ax[0,2], 'Rate [Hz]', title='Zodiacal Light', show_frame=False)
disp4 = ctapipe_disp(cam, CT1, res[res.source == type(gaia)], ax[1,0], 'Rate [Hz]', title='Bright Stars', show_frame=False)
disp5 = ctapipe_disp(cam, CT1, res[res.source == type(hmag)], ax[1,1], 'Rate [Hz]', title='Low Brightness Stars', show_frame=False)
disp6 = ctapipe_disp(cam, CT1, res[res.source == type(gbl)], ax[1,2], 'Rate [Hz]', title='Galactic Diffuse', show_frame=False)